<a href="https://colab.research.google.com/github/brianpyai/chatbot/blob/main/gemma_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install gradio transformers
!pip install huggingface_hub
#CMAKE_ARGS="-DLLAMA_BLAS=ON -DLLAMA_BLAS_VENDOR=OpenBLAS" \
!pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.9/36.9 MB 30.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.4 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.56-cp310-cp310-manylinux_2_35_x86_64.whl size=2831714 sha256=5ea4aa86abe48bb538d750b1b321692accfaf259fb9c7e21ec822b861f4d01f8
  Stored in directory: /root/.cache/pip/wheels/e5/09/9d/c413053f6258cb2546cc792418c595e276f9efd5db31a80377
Successfully built llama-cpp-python


In [ ]:

# Linux and Mac


from llama_cpp import Llama,LlamaRAMCache,LlamaDiskCache,LlamaTokenizer,LlamaState

import os
from pathlib import Path
from huggingface_hub import hf_hub_download

modelsPath="models/"
model_id = "LoneStriker/openbuddy-gemma-7b-v19.1-4k-GGUF/openbuddy-gemma-7b-v19.1-4k-Q4_K_M.gguf"
model_id="mlabonne/Gemmalpaca-2B-GGUF/gemmalpaca-2b.Q4_K_M.gguf"

def downHgFile(url,targetDir=modelsPath):
    fileName=Path (model_id ).name
    repo =model_id[:-len(fileName)-1]
    fileExists=os.path.exists(os.path.join(targetDir,fileName))

    print(repo,fileName,fileExists )
    if not fileExists:
        print ("Downloading file :")
        hf_hub_download(repo ,filename=fileName,local_dir=targetDir,)

downHgFile(model_id )
modelPath=os.path.join(modelsPath,Path(model_id).name)

model = Llama(modelPath)
tokenizer = model.tokenize




import gradio as gr
import torch

from threading import Thread




partial_message = ""
def predict(message, history):
    global partial_message
    prompt=f"""
###Below is an instruction that describes my question or task.
Write a response that appropriately completes the request:
{message}
### Response :

"""

    model_inputs = tokenizer(prompt.encode("utf-8"))
    generate_kwargs = dict(top_k=140, top_p=0.95, temp=0.95, repeat_penalty=1.1)


    partial_message = ""
    for new_token in model.generate(model_inputs,**generate_kwargs):
            try:s=model.detokenize([new_token])
            except :break
            if not s or not isinstance(s,bytes):break
            partial_message += s.decode("utf-8")
            yield partial_message

chatbot = gr.Chatbot(likeable=True,show_copy_button=True)
textbox = gr.Textbox (show_copy_button=True)
gr.ChatInterface(predict,chatbot=chatbot,textbox=textbox,stop_btn="Stop" ,retry_btn="Retry",concurrency_limit=1).launch(debug=True,share=True ,inline=False )

llama_model_loader: loaded meta data with 23 key-value pairs and 164 tensors from models/gemmalpaca-2b.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma
llama_model_loader: - kv   1:                               general.name str              = Gemmalpaca-2B
llama_model_loader: - kv   2:                       gemma.context_length u32              = 8192
llama_model_loader: - kv   3:                     gemma.embedding_length u32              = 2048
llama_model_loader: - kv   4:                          gemma.block_count u32              = 18
llama_model_loader: - kv   5:                  gemma.feed_forward_length u32              = 16384
llama_model_loader: - kv   6:                 gemma.attention.head_count u32              = 8
llama_model_loader: - kv   7:              gemma.attention.head_count_kv u3

mlabonne/Gemmalpaca-2B-GGUF gemmalpaca-2b.Q4_K_M.gguf True


llama_model_loader: - kv  13:                      tokenizer.ggml.scores arr[f32,256000]  = [0.000000, 0.000000, 0.000000, 0.0000...
llama_model_loader: - kv  14:                  tokenizer.ggml.token_type arr[i32,256000]  = [3, 3, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, ...
llama_model_loader: - kv  15:                tokenizer.ggml.bos_token_id u32              = 2
llama_model_loader: - kv  16:                tokenizer.ggml.eos_token_id u32              = 1
llama_model_loader: - kv  17:            tokenizer.ggml.unknown_token_id u32              = 3
llama_model_loader: - kv  18:            tokenizer.ggml.padding_token_id u32              = 0
llama_model_loader: - kv  19:               tokenizer.ggml.add_bos_token bool             = true
llama_model_loader: - kv  20:               tokenizer.ggml.add_eos_token bool             = false
llama_model_loader: - kv  21:               general.quantization_version u32              = 2
llama_model_loader: - kv  22:                          general.file_t

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://6f31a240e6519c6d96.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:

# Linux and Mac


from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "LoneStriker/OrcaGemma-2B-8.0bpw-h8-exl2" #@param ["LoneStriker/OrcaGemma-2B-8.0bpw-h8-exl2","LoneStriker/Gemmalpaca-2B-8.0bpw-h8-exl2","LoneStriker/zephyr-7b-gemma-v0.1-GGUF","LoneStriker/openbuddy-gemma-7b-v19.1-4k-8.0bpw-h8-exl2","LoneStriker/gemma-2b","unsloth/gemma-7b" ,"HuggingFaceH4/zephyr-7b-gemma-v0.1","openchat/openchat-3.5-0106-gemma","google/gemma-7b","google/gemma-2b","ccrains/larson-gemma-2b-chinese-v0.1" ,"abideen/gemma-2b-openhermes"]
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained("google/gemma-7b")



import gradio as gr
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer
from threading import Thread



class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        stop_ids = [29, 0]
        for stop_id in stop_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False
partial_message = ""
def predict(message, history):
    global partial_message
    prompt=f"""
###Below is an instruction that describes my question or task.
Write a response that appropriately completes the request:
{message}
### Response :

"""
    stop = StopOnTokens()

    model_inputs = tokenizer(prompt , return_tensors="pt")
    streamer = TextIteratorStreamer(tokenizer, timeout=200, skip_prompt=True , skip_special_tokens=False )
    generate_kwargs = dict(
        model_inputs,
        streamer=streamer,
        do_sample =True ,
        max_new_tokens=4096,
        top_p=0.95,
        top_k=150,
        temperature=0.95,
        stopping_criteria=StoppingCriteriaList([stop]))
    t = Thread(target=model.generate,kwargs=generate_kwargs)
    t.start()

    partial_message = ""
    for new_token in streamer:
            partial_message += new_token
            yield partial_message

chatbot = gr.Chatbot(likeable=True,show_copy_button=True)
textbox = gr.Textbox (show_copy_button=True)
gr.ChatInterface(predict,chatbot=chatbot,textbox=textbox,stop_btn="Stop" ,retry_btn="Retry",concurrency_limit=1).launch(debug=True,share=True ,inline=False )

tokenizer_config.json:   0%|          | 0.00/2.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://22c72fbbb4c3819d28.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
